Plusieurs types d'indicateurs vont être construits :

- des indicateurs de performances récentes
- des indicateurs de performances passées, sur la base des données d'historisation récupérées
- des indicateurs sur les confrontations directes entre les deux clubs
- des facteurs externes pouvant influencer le résultat d'un match (TODO)

In [1]:
import os
import sys
root_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(root_path)

import pandas as pd

from src.config import load_config
from src.preprocessing import Preprocessing
import src.utils


# config.yaml importation
config_file = 'config.yaml'
config_path = os.path.join(root_path, config_file)
config = load_config(config_path)

In [2]:
cleaned_data_path = os.path.join(root_path, config['cleaned_dir'])
test_df = pd.read_csv(cleaned_data_path+'/cleaned_df_train.csv')

In [3]:
test = Preprocessing(test_df, config)

In [4]:
#test.computes_current_season_indicators()
#test.computes_absolute_recent_form_indicators()
#test.computes_absolute_historical_form_indicators()
test.computes_strict_relative_recent_form_indicators()

,date,season,home,away,nb_goals_home,nb_goals_away,final_result,B365H,B365D,B365A,...,strict_rel_recent_nb_points_by_match_home_team,strict_rel_recent_nb_points_by_match_away_team,strict_rel_recent_nb_goals_scored_by_match_home_team,strict_rel_recent_nb_goals_scored_by_match_away_team,strict_rel_recent_nb_goals_conceded_by_match_home_team,strict_rel_recent_nb_goals_conceded_by_match_away_team,strict_rel_recent_goal_difference_home_team,strict_rel_recent_goal_difference_away_team,strict_rel_percentage_victory_home_team,strict_rel_percentage_victory_away_team
0,2010-08-07,2010/2011,Auxerre,Lorient,2.0,2.0,draw,1.91,3.30,4.20,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,2010-08-07,2010/2011,Lens,Nancy,1.0,2.0,away,2.10,3.10,3.80,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,2010-08-07,2010/2011,Lyon,Monaco,0.0,0.0,draw,1.44,4.00,8.00,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,2010-08-07,2010/2011,Marseille,Caen,1.0,2.0,away,1.25,5.50,12.00,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,2010-08-07,2010/2011,Nice,Valenciennes,0.0,0.0,draw,2.10,3.10,3.75,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5140,2024-05-19,2023/2024,Lille,Nice,2.0,2.0,draw,1.65,4.00,5.00,...,1.4,1.4,1.6,1.4,1.4,1.6,1.0,-1.0,40.0,40.0
5141,2024-05-19,2023/2024,Lens,Montpellier,2.0,2.0,draw,1.44,4.75,6.50,...,1.8,1.2,1.4,0.8,0.8,1.4,3.0,-3.0,60.0,40.0
5142,2024-05-19,2023/2024,Le Havre,Marseille,1.0,2.0,away,4.50,3.60,1.80,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
5143,2024-05-19,2023/2024,Lorient,Clermont,5.0,0.0,home,1.67,4.00,4.75,...,2.0,0.5,1.5,1.0,1.0,1.5,1.0,-1.0,50.0,0.0
